In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.8 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar 

--2022-03-06 07:29:45--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.46MB/s    in 0.2s    

2022-03-06 07:29:46 (5.46 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Demographics").getOrCreate()

In [6]:
# Read in data from S3 Buckets
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark import SparkFiles
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz'
spark.sparkContext.addFile(url)

schema = StructType([ 
    StructField("marketplace",StringType(),True), 
    StructField("customer_id",IntegerType(),True), 
    StructField("review_id",StringType(),True), 
    StructField("product_id", IntegerType(), True),
    StructField("product_parent", IntegerType(), True),
    StructField("product_title", StringType(), True),
    StructField("product_category", StringType(), True),
    StructField("star_rating", IntegerType(), True),
    StructField('helpful_vote', IntegerType(), True),
    StructField('total_votes', IntegerType(), True),
    StructField('vine', StringType(), True),
    StructField('verified_purchase', StringType(), True),
    StructField('review_headline', StringType(), True),
    StructField('review_body', StringType(), True),
    StructField('review_date', StringType(), True),
  ])

df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, schema=schema, timestampFormat="yyyy/MM/dd HH:mm:ss")

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_vote|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|      null|     937001370|Invicta Women's 1...|         Watches|          5|           0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|      null|     484010722|Kenneth Cole New ...|         Watches|          5|        

In [7]:
# total records (number of rows): 960872 rows 
df.count()

960872

In [8]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_vote: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [9]:
# review_id table
review_id_df = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_df = review_id_df.dropna()
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1HV2WX91XXH2X|   12497360|1850813491|     645298931| 2014-07-31|
|R12R22J82MGZOC|   36487387|1397458135|     262368674| 2013-05-13|
|R3UZ3JCAN5LGEZ|   51483206|1380137136|     518987366| 2013-03-14|
|R3IEK1LOS66U53|   15274551|1880241196|     582737217| 2012-12-12|
+--------------+-----------+----------+--------------+-----------+



In [10]:
# products table
products_df = df.select(['product_id', 'product_title'])
products_df = products_df.dropna()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1850813491|Casio Men's Watch...|
|1397458135|New Smart Steampu...|
|1380137136|Bronze Brass Past...|
|1880241196|Bird Watching For...|
+----------+--------------------+



In [11]:
# customers table
customer_df = df.groupBy('customer_id').count()
customer_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    1567510|    1|
|   19502021|    1|
|   12819130|    1|
|   35329257|    2|
|     108460|    1|
|    5453476|    1|
|   29913055|    1|
|   30717305|    1|
|    1570030|    1|
|   19032020|    1|
|   44178035|    1|
|   26079415|    2|
|   14230926|    1|
|   43478048|    2|
|   43694941|    1|
|   12318815|    3|
|   13731855|    1|
|     740134|    1|
|   41956754|    1|
|   20324070|    3|
+-----------+-----+
only showing top 20 rows



In [12]:
# vine table
vine_df = df.select(['review_id', 'star_rating', 'helpful_vote', 'total_votes', 'vine'])
vine_df = vine_df.dropna()
vine_df.show()

+--------------+-----------+------------+-----------+----+
|     review_id|star_rating|helpful_vote|total_votes|vine|
+--------------+-----------+------------+-----------+----+
|R3O9SGZBVQBV76|          5|           0|          0|   N|
| RKH8BNC3L5DLF|          5|           0|          0|   N|
|R2HLE8WKZSU3NL|          2|           1|          1|   N|
|R31U3UH5AZ42LL|          5|           0|          0|   N|
|R2SV659OUJ945Y|          4|           0|          0|   N|
| RA51CP8TR5A2L|          5|           0|          0|   N|
| RB2Q7DLDN6TH6|          5|           1|          1|   N|
|R2RHFJV0UYBK3Y|          1|           1|          5|   N|
|R2Z6JOQ94LFHEP|          5|           1|          2|   N|
| RX27XIIWY5JPB|          4|           0|          0|   N|
|R15C7QEZT0LGZN|          4|           2|          2|   N|
|R361XSS37V0NCZ|          1|           0|          0|   N|
| ROTNLALUAJAUB|          3|           0|          0|   N|
|R2DYX7QU6BGOHR|          5|           0|          0|   

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/classroom_database"
config = {"user":"root", 
          "password": "password", 
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table_2', mode=mode, properties=config)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table="products_2", mode=mode, properties=config)

In [ ]:
customer_df.write.jdbc(url=jdbc_url, table="customers_2", mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table_2", mode=mode, properties=config)